In [1]:
import sys
import os
# Get the parent directory
parent_dir = os.getcwd()
# Add the parent directory to sys.path
sys.path.append(parent_dir)

In [2]:
!pip install nlopt joblib

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://artifacts-read.gkp.jpmchase.net/artifactory/api/pypi/pypi/simple


In [3]:
import numpy as np
import pandas as pd
from itertools import combinations
import scipy
import qokit
from qokit import get_qaoa_objective
from qokit.parameter_utils import to_basis, from_basis
from contextlib import contextmanager
import time
from typing import Optional, Tuple, Callable
from joblib import Parallel, delayed
import nlopt

In [4]:
# Constants for the linear ramp initialization
GAMMA_START = 0.0
GAMMA_END = 0.1
BETA_START = -0.1
BETA_END = 0.0


In [5]:
# Constants
max_evals = 40000
ratio_threshold = 0.98
overlap_threshold = 0.5
rhobeg = 0.01

In [6]:
def initialize_parameters(gamma: Optional[np.ndarray], beta: Optional[np.ndarray], p: int) -> Tuple[np.ndarray, np.ndarray]:
    if gamma is None and beta is None:
        gamma = np.linspace(GAMMA_START, GAMMA_END, p)
        beta = np.linspace(BETA_START, BETA_END, p)
    elif gamma is not None and beta is not None:
        assert len(gamma) == len(beta)
    return gamma, beta

In [7]:
@contextmanager
def suppress_stdout():
    with open(os.devnull, 'w') as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout

In [8]:
def func_with_gpu_id(func):
    def wrapper(*args, gpu_id, **kwargs):
        os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)
        return func(*args, **kwargs)
    return wrapper


def parallel_execution(func, inputs, num_gpus):
    func_for_gpu = func_with_gpu_id(func)
    jobs = [delayed(func_for_gpu)(*given_input, gpu_id=i % num_gpus) for i, given_input in enumerate(inputs)]
    results = Parallel(n_jobs=num_gpus, backend="loky")(jobs)
    return results

def initialize_csv(filename: str):
    columns = ["N", "p", "num_coeffs", "gamma", "beta", "basis", "u", "v", "optimizer", "approx_ratio", "merit", "overlap", "evaluations"]
    df = pd.DataFrame(columns=columns)
    df.to_csv(filename, index=False)

def generate_p_values(p0: int, pmax: int, step: int) -> np.ndarray:
    if step > p0:
        return np.concatenate(([p0], np.arange(step, pmax + 1, step, dtype=int)))
    return np.arange(p0, pmax + 1, step, dtype=int)

def fine_tune_result(f_overlap, f, gs_energy, N, p, num_coeffs, gamma, beta, basis, optimizer="bobyqa", rhobeg=None, maxiter=1000, max_energy=0):
    res, res_gamma, res_beta = fine_tune_coeffs(
        f_overlap, f, gs_energy, N, p, num_coeffs, gamma, beta, basis, optimizer=optimizer, rhobeg=rhobeg, maxiter=maxiter, max_energy=max_energy
    )
    u, v = res["x"][:num_coeffs], res["x"][num_coeffs:]

    minus_approx_ratio = res["fun"]
    approx_ratio = -1.0 * minus_approx_ratio
    merit = max_energy - approx_ratio * (max_energy - gs_energy)
    overlap = 1 - f_overlap(np.hstack([res_gamma, res_beta]))

    nfev = res["nfev"]
    result = {
        "N": N,
        "p": p,
        "num_coeffs": num_coeffs,
        "gamma": res_gamma,
        "beta": res_beta,
        "basis": basis,
        "u": u,
        "v": v,
        "optimizer": optimizer,
        "approx_ratio": approx_ratio,
        "merit": merit,
        "overlap": overlap,
        "evaluations": nfev,
    }
    return result

def fine_tune_coeffs(f_overlap, f, gs_energy, N, p, num_coeffs, gamma, beta, basis, optimizer="bobyqa", rhobeg=None, maxiter=1000, max_energy=0):
    if rhobeg is None:
        rhobeg = 0.01 / N
    print(f"Optimizing with optimizer={optimizer} for N={N}, p={p}, num_coeffs = {num_coeffs} in {basis} basis")
    u, v = to_basis(gamma, beta, num_coeffs, basis)
    initial_pt = np.hstack([u[:num_coeffs], v[:num_coeffs]])

    def func(ins):
        u[:num_coeffs] = ins[:num_coeffs]
        v[:num_coeffs] = ins[num_coeffs:]
        gamma, beta = from_basis(u, v, p, basis)
        merit = f(np.hstack([gamma, beta]))  # this value of obj is negative
        approx_ratio = (max_energy - merit) / (max_energy - gs_energy)
        minus_approx_ratio = -1.0 * approx_ratio
        return minus_approx_ratio

    if optimizer == "bobyqa":
        res = optimize_bobyqa(func, initial_pt, rhobeg=rhobeg, tol=1e-6, maxiter=maxiter)
    elif optimizer == "cobyla":
        res = scipy.optimize.minimize(func, initial_pt, method="COBYLA", options={"rhobeg": rhobeg, "tol": 1e-6, "maxiter": maxiter})
    elif optimizer == "diff_evo":
        # Here maxiter are the number of generations so there number must be set by total function evals
        # The maximum number of function evaluations (with no polishing) is: (generations + 1) * popsize * 2 * num_coeffs
        # default popsize is 15
        # Note that bound is not strict since it does take into account polishing
        popsize = 15
        generations = maxiter // (popsize * 2 * num_coeffs) - 1
        bounds = [(-0.5, 0.5)] * (2 * num_coeffs)
        res = scipy.optimize.differential_evolution(func, bounds=bounds, maxiter=generations, popsize=popsize)
    elif optimizer == "dual_anneal":
        # Here maxfun are the number of evaluations so there number must be set by total function evals
        # Note that bound is not strict since it does take into account that
        # optimization cannot be stopped during a local search
        bounds = [(-0.5, 0.5)] * (2 * num_coeffs)
        res = scipy.optimize.dual_annealing(func, bounds=bounds, maxfun=maxiter)

    print(res)
    res_gamma, res_beta = from_basis(u, v, p, basis)
    return res, res_gamma, res_beta

def optimize_bobyqa(func, params, rhobeg=None, tol=None, maxiter=3000):
    def wrapped_func(x, grad):
        if grad.size > 0:
            grad[:] = np.zeros_like(x)
        return func(x)

    opt = nlopt.opt(nlopt.LN_BOBYQA, len(params))

    if rhobeg is not None:
        opt.set_initial_step(rhobeg)
    if tol is not None:
        opt.set_ftol_rel(tol)
    if maxiter is not None:
        opt.set_maxeval(maxiter)

    opt.set_min_objective(wrapped_func)

    lower_bounds = np.full(len(params), -10)
    upper_bounds = np.full(len(params), 10)

    opt.set_lower_bounds(lower_bounds)
    opt.set_upper_bounds(upper_bounds)

    optimized_params = opt.optimize(params)
    minf = opt.last_optimum_value()
    nfev = opt.get_numevals()
    result_code = opt.last_optimize_result()

    if result_code == 1:
        message = "Optimization terminated successfully."
        success = "True"
    if result_code == 3:
        message = "Optimization stopped because ftol was reached."
        success = "True"
    elif result_code == 5:
        message = "Maximum number of function evaluations has been exceeded."
        success = "False"

    res = {
        "message": message,
        "success": success,
        "status": result_code,
        "fun": minf,
        "x": optimized_params,
        "nfev": nfev,
        "maxcv": 0.0,
    }
    return res

def save_result(result, filename):
    df = pd.DataFrame([result])
    np.set_printoptions(threshold=sys.maxsize)  # outputs with length > 1000 are also printed in full
    df.to_csv(filename, mode="a", header=False, index=False)

def log_walltime(start_time: float):
    walltime_seconds = time.time() - start_time
    walltime_hours = int(walltime_seconds // 3600)
    walltime_minutes = (walltime_seconds % 3600) / 60
    print(f"Walltime: {walltime_hours} hrs and {walltime_minutes:.2f} minutes")



In [9]:
def II_schedule(
    f_overlap: Callable[[np.ndarray], float],
    f: Callable[[np.ndarray], float],
    gs_energy: float,
    N: int,
    pmax: int = 100,
    step: int = 5,
    epsilon: float = 0.01,
    patience: int = 5,
    filename: str = None,
    p0: int = 5,
    gamma: Optional[np.ndarray] = None,
    beta: Optional[np.ndarray] = None,
    save: bool = True,
    basis: str = 'chebyshev',
    optimizer: str = 'bobyqa',
    maxiter: int = 1000,
    overlap_conv: bool = True,
    max_energy: float = 0,
    ratio_threshold: float = 0.98,
    rhobeg: float = 0.01
) -> dict:
    start_time = time.time()
    total_eval = 0
    
    if filename is None:
        filename = f'II_N_{N}_pmax_{pmax}_step_{step}_optimizer_{optimizer}.csv'

    if save and filename and not os.path.exists(filename):
        initialize_csv(filename)

    gamma, beta = initialize_parameters(gamma, beta, p=p0)

    if p0 < 5:
        init_p = np.array([1,2, 3, 4])
        p_values = np.concatenate((init_p, generate_p_values(5, pmax, step)))
        
    else:
        p_values = generate_p_values(p0, pmax, step)
        
    num_coeffs = 5
    
    counter = 0
    prev_approx_ratio = 0
    print(f"Optimal value for N={N} is {gs_energy}")

    for p in p_values:
        if p<=5:
            num_coeffs = p
        
        u, v = to_basis(gamma, beta, num_coeffs, basis)
        gamma, beta = from_basis(u, v, p, basis)
        result = fine_tune_result(f_overlap, f, gs_energy, N, p, num_coeffs, gamma, beta, basis, optimizer, rhobeg, maxiter, max_energy)
        gamma, beta = result['gamma'], result['beta']
        approx_ratio = result['approx_ratio']
        overlap = result['overlap']
        total_eval += result['evaluations']

        if save and filename:
            save_result(result, filename)

        if overlap_conv==True:
            if overlap > overlap_threshold or total_eval > max_evals:
                break
                
        if overlap_conv==False:
            if approx_ratio > ratio_threshold or total_eval > max_evals:
                break

        improvement = (approx_ratio - prev_approx_ratio) / approx_ratio
        if improvement < epsilon:
            counter += 1

        if counter == patience:
            num_coeffs += 1
            counter = 0

        prev_approx_ratio = approx_ratio

    log_walltime(start_time)
    print(f"Total evaluations: {total_eval}")
    return result

In [10]:
def gs(N, terms):
    simclass = qokit.fur.choose_simulator(name='auto')
    sim = simclass(N, terms=terms)
    sorted_diag = np.sort(sim.get_cost_diagonal())
    gs = sorted_diag[0]
    return gs

In [11]:
def createList(r1, r2):
    return [item for item in range(r1, r2+1)]

In [12]:
def run_trial(N, trial, base_seed, pmax, step, optimizer, dir_name):
    with suppress_stdout():
        seed = base_seed + trial
        np.random.seed(seed)
        terms = [(np.random.normal(), spin_pair) for spin_pair in combinations(range(N), r=2)]
        
        f_overlap = get_qaoa_objective(N, terms=terms, objective="overlap")
        f = get_qaoa_objective(N, terms=terms, objective="expectation")
        
        gs_energy = gs(N, terms)
        
        filename = os.path.join(dir_name, f"II_N_{N}_SK_results_seed_{seed}_pmax_{pmax}_step_{step}_optimizer_{optimizer}.csv")

        result = II_schedule(
            f_overlap=f_overlap,
            f=f,
            gs_energy=gs_energy,
            N=N,
            pmax=pmax,
            step=step,
            save=True,
            filename=filename,
            p0=5,
            overlap_conv=True
        )

In [13]:
def main(N):
    base_seed = 0
    num_trials = 5
    pmax = 100
    step = 5
    optimizer = 'bobyqa'
    num_gpus = 4
    dir_name = f"ii_results/SK_N{N}"
    os.makedirs(dir_name, exist_ok=True)
    
    inputs = [(N, trial, base_seed, pmax, step, optimizer, dir_name) for trial in range(num_trials)]
    
    results = parallel_execution(run_trial, inputs, num_gpus)

In [14]:
N = 6
main(N)

brajesh-7c968f6698-5vms7:pid984: PSM3 can't open nic unit: 0 (err=23)
brajesh-7c968f6698-5vms7:pid984: PSM3 can't open nic unit: 0 (err=23)
brajesh-7c968f6698-5vms7:pid984: PSM3 can't open nic unit: 0 (err=23)
brajesh-7c968f6698-5vms7:pid985: PSM3 can't open nic unit: 0 (err=23)
brajesh-7c968f6698-5vms7:pid985: PSM3 can't open nic unit: 0 (err=23)
brajesh-7c968f6698-5vms7:pid985: PSM3 can't open nic unit: 0 (err=23)
brajesh-7c968f6698-5vms7:pid986: PSM3 can't open nic unit: 0 (err=23)
brajesh-7c968f6698-5vms7:pid986: PSM3 can't open nic unit: 0 (err=23)
brajesh-7c968f6698-5vms7:pid986: PSM3 can't open nic unit: 0 (err=23)
brajesh-7c968f6698-5vms7:pid983: PSM3 can't open nic unit: 0 (err=23)
brajesh-7c968f6698-5vms7:pid983: PSM3 can't open nic unit: 0 (err=23)
brajesh-7c968f6698-5vms7:pid983: PSM3 can't open nic unit: 0 (err=23)
brajesh-7c968f6698-5vms7:pid984: brajesh-7c968f6698-5vms7:pid983: PSM3 can't open nic unit: 0 (err=23)PSM3 can't open nic unit: 0 (err=23)

brajesh-7c968f6698-5

brajesh-7c968f6698-5vms7:pid984.python: Failed to get eth0 (unit 0) cpu set
brajesh-7c968f6698-5vms7:pid984.python: Failed to get eth0 (unit 0) cpu set
brajesh-7c968f6698-5vms7:pid984.python: Failed to get eth0 (unit 0) cpu set
brajesh-7c968f6698-5vms7:pid985.python: Failed to get eth0 (unit 0) cpu set
brajesh-7c968f6698-5vms7:pid985.python: Failed to get eth0 (unit 0) cpu set
brajesh-7c968f6698-5vms7:pid985.python: Failed to get eth0 (unit 0) cpu set
brajesh-7c968f6698-5vms7:pid986.python: Failed to get eth0 (unit 0) cpu set
brajesh-7c968f6698-5vms7:pid986.python: Failed to get eth0 (unit 0) cpu set
brajesh-7c968f6698-5vms7:pid986.python: Failed to get eth0 (unit 0) cpu set
brajesh-7c968f6698-5vms7:pid983.python: Failed to get eth0 (unit 0) cpu set
brajesh-7c968f6698-5vms7:pid983.python: Failed to get eth0 (unit 0) cpu set
brajesh-7c968f6698-5vms7:pid983.python: Failed to get eth0 (unit 0) cpu set
brajesh-7c968f6698-5vms7:pid984.python: Failed to get eth0 (unit 0) cpu set
brajesh-7c96

/efsMount/R753585/brajesh/qiskit2/lib64/python3.11/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/efsMount/R753585/brajesh/qiskit2/lib64/python3.11/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/efsMount/R753585/brajesh/qiskit2/lib64/python3.11/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/efsMount/R753585/brajesh/qiskit2/lib64/python3.11/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/efsMount/R753585/brajesh/qiskit2/lib64/python3.11/site-pack